In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-support/Cust_1k.json
/kaggle/input/customer-support-data/Cust_1k.json


In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers trl==0.4.7

In [4]:
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch


2024-05-04 20:00:44.789335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 20:00:44.789466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 20:00:44.926207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")
!huggingface-cli login --token $secret_value_0


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import accelerate
#model = AutoModelForCausalLM.from_pretrained("PygmalionAI/pygmalion-6b", low_cpu_mem_usage=True, device_map='auto', torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("uday91/Meta-Llama-3-8B-customer-support-finetune", device_map='auto', torch_dtype=torch.float16)

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("uday91/Meta-Llama-3-8B-customer-support-finetune")


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:

# # Example prompt
# prompt = "help to see when my purchase is going to arrive"

# # Generate text
# input_ids = tokenizer.encode(prompt, return_tensors="pt")
# output = model.generate(input_ids, max_length=100, num_return_sequences=1)
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# print("Generated text:")
# print(generated_text)


In [9]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [10]:

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "help to see when my purchase is going to arrive?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] help to see when my purchase is going to arrive? [/INST] I understand your curiosity about the estimated arrival time of your purchase. To provide you with accurate information, could you please provide me with the {{Order Number}} or {{Tracking Number}}? With that, I'll be able to track your order and provide you with the most up-to-date information regarding its delivery. Thank you for your cooperation! </s> How can I assist you further? " </s> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </inst> </i

In [11]:

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "how to edit my delivery address "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] how to edit my delivery address  [/INST] I appreciate your message to me about editing your delivery address. I'm here to assist you with this process. To edit your delivery address, you can follow these steps:

1. Log in to your account on our website.
2. Go to the "My Account" or "Profile" section.
3. Look for the "Shipping Addresses" or similar option.
4. Click on the "Edit" or "Manage" button next to the address you want to update.
5. Make the necessary changes to your delivery address.
6. Save the updated information.

If you encounter any difficulties or have any questions during the process, please don't hesitate to let me know. I'm here to help you every step of the way. Thank you for trusting us with your delivery address details. We value your feedback and appreciate your patience as we assist you with this important update. </s> How can I assist you further? " </s> Thank you for reaching out to us! I understand that you would like to edit your delivery address. To 

In [11]:

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "how to check the payment modalities?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] how to check the payment modalities? [/INST] I understand your need to check the payment modalities available to you. Here's some information about the different payment options:

1. **Credit/Debit Card:** You can use a credit or debit card to make payments securely online.
2. **PayPal:** If you have a PayPal account, you can use it to pay for your purchases.
3. **Bank Transfer:** You can also make payments through your bank account using the bank transfer option.
4. **Cash on Delivery:** If you prefer, you can choose the cash on delivery option and pay in cash when your order arrives.

If you have any specific questions about a particular payment method or need assistance with any of these options, please let me know. I'm here to help! </s> How can I assist you further? " Check out our payment modalities at https://www.website.com/payment-modalities. If you have any questions or need further assistance, feel free to reach out. I'm here to help! " How can I assist you further

In [13]:

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Considering leaving a review for a service I've experienced."
#pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Considering leaving a review for a service I've experienced. [/INST] I appreciate your interest in leaving a review for our service. Your feedback is incredibly valuable to us as it helps us understand how we can improve and provide better experiences for our customers. To leave a review, please visit our website and navigate to the "Reviews" section. There, you will find a form where you can share your thoughts and experiences. We encourage you to be as detailed as possible, so that we can gain a deeper understanding of your experience. Thank you for taking the time to share your feedback with us, and we look forward to hearing from you! </s> I'm here to assist you with leaving a review for our service. To get started, please provide me with the details of your experience, including the specific service you would like to review and any feedback you have. This information will help me guide you through the process and ensure that your review is accurate and informative. Thank

In [14]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Considering leaving a review for a service I've experienced."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000,temperature =0.5)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Considering leaving a review for a service I've experienced. [/INST] I'm delighted to assist you with leaving a review for a service you've experienced. Your feedback is highly valuable to us, as it helps us improve our services and provide better experiences for our customers. Could you please provide me with more details about the specific service you would like to review? This will enable me to guide you through the process and ensure that your review is accurately reflected. </s> </INST> "Consider leaving a review" is a phrase that indicates a person is contemplating leaving a review for a service they have experienced. To provide the best assistance, I would appreciate it if you could provide me with more details about the specific service you are thinking of reviewing. This will enable me to guide you through the process and ensure that your review is accurately reflected. Please let me know which service you are considering leaving a review for, and I will be more than

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
hf_model_id = "uday91/Meta-Llama-3-8B-customer-support-finetune"
model = AutoModelForCausalLM.from_pretrained(hf_model_id,device_map='auto', torch_dtype=torch.float16)


config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(hf_model_id, legacy=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
generation_config, unused_kwargs = GenerationConfig.from_pretrained(
    hf_model_id, max_new_tokens=300, temperature=0.3, return_unused_kwargs=True
)
model.generation_config = generation_config
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,

)
prompt = "Considering leaving a review for a service I've experienced."

result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Considering leaving a review for a service I've experienced. [/INST] I'm aware that you're considering leaving a review for a service you've experienced. Your feedback is valuable to us, and we appreciate your willingness to share your thoughts. If you have any specific questions or need assistance with the review process, please let me know. I'm here to help you every step of the way. </s> " I'm aware that you're considering leaving a review for a service you've experienced. Your feedback is valuable to us, and we appreciate your willingness to share your thoughts. If you have any specific questions or need assistance with the review process, please let me know. I'm here to help you every step of the way. " </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </

In [9]:
generation_config, unused_kwargs = GenerationConfig.from_pretrained(
    hf_model_id, max_new_tokens=500, temperature=0.5, return_unused_kwargs=True
)
model.generation_config = generation_config
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,

)
prompt = "Considering leaving a review for a service I've experienced."

result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Considering leaving a review for a service I've experienced. [/INST] I'm glad you're considering leaving a review for our service. Your feedback is incredibly valuable to us as it helps us understand how we can improve and provide you with the best experience possible. If you have any specific details about the service you would like to share, please feel free to elaborate. Your honest thoughts and opinions will help us make positive changes and ensure that we continue to meet your expectations. Thank you for taking the time to share your experience with us. </s> If you have any other questions or need further assistance, please don't hesitate to let me know. I'm here to help you every step of the way. " </s> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST> </INST>

In [11]:
generation_config, unused_kwargs = GenerationConfig.from_pretrained(
    hf_model_id, max_new_tokens=500, temperature=0.5, return_unused_kwargs=True
)
model.generation_config = generation_config
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,

)
prompt = "Considering leaving a review for a service I've experienced."

result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Considering leaving a review for a service I've experienced. [/INST] I'm so glad you're considering leaving a review for our service! Your feedback is incredibly valuable to us as it helps us improve and provide better experiences for our customers. If you could share your thoughts and experiences with our service, it would be greatly appreciated. You can leave a review on our website, social media platforms, or through our customer support team. We're here to listen and learn from your experiences. Thank you for your contribution! </s> " I'm so glad you're considering leaving a review for our service! Your feedback is incredibly valuable to us as it helps us improve and provide better experiences for our customers. If you could share your thoughts and experiences with our service, it would be greatly appreciated. You can leave a review on our website, social media platforms, or through our customer support team. We're here to listen and learn from your experiences. Thank you

In [15]:
generation_config, unused_kwargs = GenerationConfig.from_pretrained(
    hf_model_id, max_new_tokens=500, temperature=0.5,return_unused_kwargs=True
)
model.generation_config = generation_config
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,

)
prompt = "Considering leaving a review for a service I've experienced."

result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Considering leaving a review for a service I've experienced. [/INST] I'm here to assist you with leaving a review for the service you've experienced. Your feedback is highly valued, and it helps us improve our services and meet your expectations. Could you please provide me with more details about the service you would like to review? This will allow me to guide you through the process and ensure that your review is accurate and informative. </s> "I'm here to assist you with leaving a review for the service you've experienced. Your feedback is highly valued, and it helps us improve our services and meet your expectations. Could you please provide me with more details about the service you would like to review? This will allow me to guide you through the process and ensure that your review is accurate and informative. </s> "I'm here to assist you with leaving a review for the service you've experienced. Your feedback is highly valued, and it helps us improve our services and m

In [14]:
generation_config, unused_kwargs = GenerationConfig.from_pretrained(
    hf_model_id, max_new_tokens=500, temperature=0.5, return_unused_kwargs=True
)
model.generation_config = generation_config
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,

)
prompt = "i want to write an review about a service"

result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 119.06 MiB is free. Process 2368 has 14.63 GiB memory in use. Of the allocated memory 13.53 GiB is allocated by PyTorch, and 993.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF